In [1]:
# https://github.com/llSourcell/pytorch_in_5_minutes/blob/master/demo.py

import torch
from torch.autograd import Variable
import pandas as pd
from random import randint
import numpy as np
import torch.nn as nn
# from torchviz import make_dot # used for grad visualization
import torch.optim as optim

In [2]:
from torch.autograd import Function
from torch.nn.modules.distance import PairwiseDistance

class TripletLoss(Function):
    
    def __init__(self, alpha):
        super(TripletLoss, self).__init__()
        self.alpha = alpha
        self.pdist  = PairwiseDistance(2)
        
    def forward(self, anchor, positive, negative):
        pos_dist   = self.pdist.forward(anchor, positive)#.pow(2)
        neg_dist   = self.pdist.forward(anchor, negative)#.pow(2)
        hinge_dist = torch.clamp(self.alpha + pos_dist - neg_dist, min = 0.0)
        loss       = torch.mean(hinge_dist)
        return loss

In [25]:
%%time
#read in relevant data
# trainData = torch.from_numpy(np.loadtxt('data/trainData.txt', dtype=np.float32))
# queryData = torch.from_numpy(np.loadtxt('data/queryData.txt', dtype=np.float32))
# df =  pd.read_pickle("./data/KNN.pkl")
df = pd.read_csv('data/KNN.csv', index_col=0) 

CPU times: user 4.11 s, sys: 373 ms, total: 4.49 s
Wall time: 4.56 s


In [5]:
def generateTripplet(index):
    point = queryData[index].reshape(-1, 1)
#     pos = trainData[df.iloc[index].KNN[randint(0,K)]].reshape(-1, 1) # pos fom KNN
#     negIndicies = list(range(K,K + 10)) + list(range(df.shape[0]-10, df.shape[0]-1))
#     neg = trainData[df.iloc[index].KNN[np.random.choice(negIndicies)]].reshape(-1, 1)
#     neg = trainData[df.iloc[index].KNN[randint(K, df.shape[0]-1)]].reshape(-1, 1)
    pos = trainData[df.iloc[index].KNN[0]].reshape(-1, 1) # pos fom KNN
    neg = trainData[df.iloc[index].KNN[5004]].reshape(-1, 1)
    return point, pos, neg

In [6]:
sigmoid = nn.Sigmoid()
def forward_pass(query):
    return sigmoid(torch.norm(query.t() - anchors, 2, 1).reshape(-1, 1) - biases)

In [7]:
BATCH_SIZE, INPUT_D, HIDDEN_D, OUTPUT_D = 32, 192, 128, 128
ALPHA = 0.5
LEARNING_RATE = 1e-2
K = 1

def init_model():
    print("--- Initialising Model Params --- ")
    
    anchors = Variable(torch.randn(OUTPUT_D, INPUT_D).type(torch.FloatTensor), requires_grad=True)
    # weights = Variable(torch.randn(HIDDEN_D, OUTPUT_D).type(torch.FloatTensor), requires_grad=True)

    # set biases to be mean value
    aggregate = torch.zeros(OUTPUT_D)
    for point in queryData:
        w0 = torch.norm(point.t() - anchors, 2, 1)
        aggregate += w0

    biases = Variable((aggregate/queryData.shape[0]).reshape(-1, 1), requires_grad=True)
    print("--- Done. Begining training ---")
    return anchors, biases

In [9]:
#training
anchors, biases = init_model()
optimizer = optim.Adam([anchors, biases], lr=LEARNING_RATE)

def train():
    for epoch in range(10000):

        #generate batch and compute collective loss for batch
        # UNSTABLE LEARNING WHEN SAMPLE SIZE > BATCH SIZE
        batch_indicies = np.random.choice(queryData.shape[0], BATCH_SIZE, replace=False) 

        loss = 0
        for index in batch_indicies:
            query, pos, neg = generateTripplet(index)
            queryMapped, posMapped, negMapped = [forward_pass(x) for x in [query, pos, neg]]
            triplet_loss = TripletLoss(ALPHA).forward(queryMapped, posMapped, negMapped)
            loss += triplet_loss

        loss /= BATCH_SIZE # computes mean so learning rate remains the same

        print(epoch, loss.data)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train()

--- Initialising Model Params --- 
--- Done. Begining training ---
0 tensor(0.5515)
1 tensor(0.5330)
2 tensor(0.4903)
3 tensor(0.4662)
4 tensor(0.5382)
5 tensor(0.4893)
6 tensor(0.5143)
7 tensor(0.5322)
8 tensor(0.5011)
9 tensor(0.4959)
10 tensor(0.5773)
11 tensor(0.5895)
12 tensor(0.4646)
13 tensor(0.5448)
14 tensor(0.4542)
15 tensor(0.4865)
16 tensor(0.5417)
17 tensor(0.5510)
18 tensor(0.5283)
19 tensor(0.4010)
20 tensor(0.5303)
21 tensor(0.5476)
22 tensor(0.4685)
23 tensor(0.5167)
24 tensor(0.5224)
25 tensor(0.4803)
26 tensor(0.4964)
27 tensor(0.5786)
28 tensor(0.5564)
29 tensor(0.4777)
30 tensor(0.5093)
31 tensor(0.5122)
32 tensor(0.5343)
33 tensor(0.4768)
34 tensor(0.5135)
35 tensor(0.4823)
36 tensor(0.4359)
37 tensor(0.4699)
38 tensor(0.5445)
39 tensor(0.5963)
40 tensor(0.4577)
41 tensor(0.5429)
42 tensor(0.5708)
43 tensor(0.4736)


KeyboardInterrupt: 